<a href="https://colab.research.google.com/github/mautovhof/Clustering-Textual-Data-with-Word2Vec/blob/main/Google_Colab_3_2_Marta_Tovias_TFM_2022_WordEmdedding_Glove_Own_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Sep  3 03:26:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [1]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pytds"] SQLAlchemy sqlalchemy-pytds

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.6 MB/s 
     |████████████████████████████████| 4.1 MB 70.5 MB/s 
     |████████████████████████████████| 55 kB 4.8 MB/s 
  Created wheel for python-tds: filename=python_tds-1.11.0-py3-none-any.whl size=69953 sha256=2eaaca4dd660e7a139f502d0737b197dd97bdf6fb99c47166b81e0fb1f5cb314
  Stored in directory: /root/.cache/pip/wheels/83/3f/26/3701ff734ec19c316df513202ba322ce6b3da58e41bfe9725a
Successfully built python-tds


In [2]:
!pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 8.0 MB/s 


In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
import os.path
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine


In [5]:
def sqlcol(dfparam):
    dtypedict = {}
    for i, j in zip(dfparam.columns, dfparam.dtypes):
        if "object" in str(j):
            dtypedict.update({i: sqlalchemy.types.NVARCHAR(length=255)})
        if "datetime" in str(j):
            dtypedict.update({i: sqlalchemy.types.DateTime()})
        if "float" in str(j):
            dtypedict.update({i: sqlalchemy.types.Float(precision=6, asdecimal=True)})
        if "int" in str(j):
            dtypedict.update({i: sqlalchemy.types.INT()})
    return dtypedict


In [6]:
# initialize parameters
project_id="infra-analyzer-317122"
region="us-central1"
instance_name="sql-marta-tovias"

INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "sqlserver"
DB_PASS = "marta.1996.1"
DB_NAME = "marta-tweets"

Your instance connection name is: infra-analyzer-317122:us-central1:sql-marta-tovias


In [7]:
from google.cloud.sql.connector import Connector
import sqlalchemy
from sqlalchemy import Column, Float, Integer, String, Table

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pytds",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mssql+pytds://localhost",
    creator=getconn,
)

In [8]:

# connect to connection pool
with pool.connect() as db_conn:
    # query database and fetch results
    cursor = db_conn.execute("SELECT top 10 ltrim(Tweet_Cleaned) as Tweet_Text from RTweets_Grouped").fetchall()


# cleanup connector
connector.close()

In [9]:

print("Tweeting")
my_list=[]
for row in cursor:
    tweet = str(row.Tweet_Text)
    my_list.append([tweet])

Tweeting


In [10]:
import glove

In [11]:
from glove import Glove
from glove import Corpus

In [12]:
glove = Glove.load('glove.model')

FileNotFoundError: ignored

In [ ]:
ww=[]
vv=[]

  for doc in my_list:
      words=doc.split()
      for w in words:
        if w != ' ':
            try:
                if len(model.get(w)>0):
                    ww.append(w)
                    vv.append(model.get(w))
            except:
                pass
  print("Generating DataFrame to Insert")

In [ ]:
print("Tokenizing")
segment=[]

for doc in lines:
    words=nlp(doc[0])
    word_list=[]
    for w in words:
        if (w.text != ' ') & (len(w.text)>3):
            word_list.append(w.lemma_)
    segment.append(word_list)

Tokenizing


In [ ]:
print(glove.word_vectors[glove.dictionary['mal']])

[ 4.24347513e-03 -2.85215070e-04  2.14450894e-02 -2.23681200e-02
 -3.46643890e-02  1.17016710e-02  2.27116041e-03 -3.75142623e-02
  1.85080058e-03 -6.59323362e-03 -6.39381956e-05 -7.30140317e-03
 -7.61475499e-03 -2.44741192e-04  1.25723883e-03  6.21176089e-03
  5.18009957e-03 -1.64182030e-02  1.22078448e-02 -1.33370441e-03
 -3.89341290e-05 -4.37640079e-02  5.71971678e-03  4.72969177e-03
  4.09246983e-03 -5.12284838e-05  6.00217035e-03  1.46867408e-02
  1.61846221e-02 -5.59331429e-03  7.77183614e-03 -2.64545484e-03
 -8.13183518e-03 -3.19364352e-02 -4.18326472e-02  4.88889228e-05
 -1.18251304e-03  5.31086240e-03 -2.98587902e-02  4.93016031e-03
 -1.39947132e-02  4.63620128e-03 -6.93766907e-03  1.73637862e-02
 -1.19081521e-02 -3.67334614e-03 -3.87715398e-02  1.23755071e-03
  3.15271231e-02 -3.38052771e-02  6.85757005e-03  8.00240908e-03
 -1.14956352e-03  3.65662448e-04  2.84557809e-03  3.92521905e-03
  9.87227305e-03  1.27080628e-03 -8.55577421e-03  3.42578223e-03
 -4.45680993e-03  3.16999

In [ ]:
glove = Glove.load('glove.model')
